In [ ]:
df <- arrow::read_parquet(here("data", "featurefull.parquet"))
df %>% glimpse()

Rows: 17,926
Columns: 67
$ winner.name                     <chr> "Shuichi Sekiguchi", "Shuichi Sekiguch…
$ winner.id                       <chr> "sl92", "sl92", "sl92", "sl92", "tf58"…
$ winner.rank                     <int> 399, 399, 399, 399, 619, 619, 619, 619…
$ winner.bornAt                   <chr> "USA", "USA", "USA", "USA", "Japan", "…
$ match.domHands                  <fct> Right-Handed_Left-Handed, Right-Handed…
$ match.backHands                 <fct> Two-Handed Backhand_Two-Handed Backhan…
$ winner.height                   <dbl> 168, 168, 168, 168, 170, 170, 170, 170…
$ looser.name                     <chr> "Yuta Shimizu", "Yuta Shimizu", "Yuta …
$ looser.id                       <chr> "s0aq", "s0aq", "s0aq", "s0aq", "s0ld"…
$ looser.rank                     <int> 464, 464, 464, 464, 1057, 1057, 1057, …
$ looser.bornAt                   <chr> "Japan", "Japan", "Japan", "Japan", "J…
$ looser.height                   <dbl> 163, 163, 163, 163, 163, 163, 163, 163…
$ match.tournam

In [ ]:
# vfold
set.seed(1)
# df.10fold <- vfold_cv(dfF, v = 5, repeats = 5)